In [1]:
import requests

In [3]:
#  Authentication is enabled and cluster connections are encrypted.

# ℹ️  Password for the elastic user (reset with `bin/elasticsearch-reset-password -u elastic`):
#   UhJ=sDusoDu=8a*JJ-H6

# ℹ️  HTTP CA certificate SHA-256 fingerprint:
#   3e100f2162f9c96b65a6c2ad7494b3646a476ea418b7f583d3e107692af47f9d

# ℹ️  Configure Kibana to use this cluster:
# • Run Kibana and click the configuration link in the terminal when Kibana starts.
# • Copy the following enrollment token and paste it into Kibana in your browser (valid for the next 30 minutes):
#   eyJ2ZXIiOiI4LjguMSIsImFkciI6WyIxNzIuMTguMC4yOjkyMDAiXSwiZmdyIjoiM2UxMDBmMjE2MmY5Yzk2YjY1YTZjMmFkNzQ5NGIzNjQ2YTQ3NmVhNDE4YjdmNTgzZDNlMTA3NjkyYWY0N2Y5ZCIsImtleSI6IjRES0JyNGdCWV94OU1qaXBaWkNHOlE5OTBwczVWVHItdUZMR3htdDFIRlEifQ==

# ℹ️ Configure other nodes to join this cluster:
# • Copy the following enrollment token and start new Elasticsearch nodes with `bin/elasticsearch --enrollment-token <token>` (valid for the next 30 minutes):
#   eyJ2ZXIiOiI4LjguMSIsImFkciI6WyIxNzIuMTguMC4yOjkyMDAiXSwiZmdyIjoiM2UxMDBmMjE2MmY5Yzk2YjY1YTZjMmFkNzQ5NGIzNjQ2YTQ3NmVhNDE4YjdmNTgzZDNlMTA3NjkyYWY0N2Y5ZCIsImtleSI6IjRqS0JyNGdCWV94OU1qaXBaWkNKOkwyRGZ5RnM4UVNTbGZLWVp1NVI0N1EifQ==


## new elasticsearch

In [4]:
from elasticsearch import Elasticsearch, helpers



es = Elasticsearch("http://localhost:9400",basic_auth=("elastic","secret"),verify_certs=False)
es.indices.get_alias(index="*")

ObjectApiResponse({'achievements': {'aliases': {}}, 'data_description': {'aliases': {}}, 'education': {'aliases': {}}, 'activity': {'aliases': {}}, 'log': {'aliases': {}}, 'appsettings': {'aliases': {}}, 'exercise': {'aliases': {}}, 'baseline': {'aliases': {}}, 'plan': {'aliases': {}}, 'account': {'aliases': {}}, 'education_queue': {'aliases': {}}})

In [ ]:
demographics={}
print(es.search(index="baseline", query={'match' : {"userid":"user"}})["hits"]["hits"][0]["_source"])
res=es.search(index="baseline", query={'match' : {"userid":"user"}})["hits"]["hits"]["_source"]["questionnaire"]

demographics["age"]=res["Dem_age"]
demographics["gender"]=res["Dem_gender"]
demographics["height"]=res["Dem_height"]
demographics["weight"]=res["Dem_weight"]
demographics

In [5]:
es.indices.exists(index="plan")

HeadApiResponse(True)

In [5]:
es.options(ignore_status=[400,404]).indices.delete(index='plan')

ObjectApiResponse({'error': {'root_cause': [{'type': 'index_not_found_exception', 'reason': 'no such index [plan]', 'resource.type': 'index_or_alias', 'resource.id': 'plan', 'index_uuid': '_na_', 'index': 'plan'}], 'type': 'index_not_found_exception', 'reason': 'no such index [plan]', 'resource.type': 'index_or_alias', 'resource.id': 'plan', 'index_uuid': '_na_', 'index': 'plan'}, 'status': 404})

In [7]:
def get_user(username):
    res = es.search(index="account", query={'match' : {"_id":username}})
    if not res["hits"]["hits"]:
        return False
    return res.body["hits"]["hits"][0]["_source"]

get_user("stuartadmin")

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'userid': 'stuartgo',
 'password': '$2b$12$phrVybcm8uyeTl9D/cdYoeZxiOHsjddjitMoYWv8lMVu9bMuY1L2a',
 'country': 'nl',
 'clinician': 'NTNU',
 'rights': ['ROLE_ADMIN', 'ACTUATOR'],
 'isaccountnonexpired': True,
 'isaccountnonlocked': True,
 'iscredentialsnonexpired': True,
 'isenabled': True}

In [9]:
res = es.search(index="data_description", query={
                                                # 'bool' : {
                                                #             "filter":{
                                                                    "terms":{
                                                                            "educationid":["Cause of LBP_3"]
                                                                            }
                                                        #             }
                                                        #     }
                                                },size=100)
res.body

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [17]:
res = es.search(index="data_description", query={
                                                # 'bool' : {
                                                #             "filter":{
                                                                    "terms":{
                                                                            "educationid":["Cause of LBP_1\xa0"]
                                                                            }
                                                        #             }
                                                        #     }
                                                },size=100)
res.body

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [20]:
res = es.search(index="data_description", query={'match' : {"educationid":"Cause of LBP_1\xa0"}},size=1)
res.body

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 11, 'relation': 'eq'},
  'max_score': 8.612359,
  'hits': [{'_index': 'data_description',
    '_id': 'rzy6LI4Bhva5Q5uik2P-',
    '_score': 8.612359,
    '_ignored': ['long_message.keyword'],
    '_source': {'headcomponent': 'Generell informasjon om ryggsmerter',
     'educationid': 'Cause of LBP_1\xa0',
     'short_message': 'Rygg- og nakkesmerter har som regel ikke en kjent årsak. Det er derfor svært lite sannsynlig at smertene dine er forårsaket av en alvorlig sykdom eller skade.',
     'link_to_long_message': 'Lær mer',
     'long_message': 'De aller fleste tilfeller av rygg- og nakkesmerter er uspesifikke. Dette betyr at smerten kommer fra muskler og ledd, men at det er vanskelig å identifisere den nøyaktige kilden til problemet. Dette er gode nyheter - du har ingen alvorlig sykdom eller skade i ryggen eller nakken din. ',
     'link_to_tool': 'Lær mer

In [12]:
res = es.search(index="data_description", query={'match' : {"description_type":"education"}},size=100)
res.body

# list(map(lambda x: x["_id"],res["hits"]["hits"]))

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 184, 'relation': 'eq'},
  'max_score': 0.6544683,
  'hits': [{'_index': 'data_description',
    '_id': 'rzy6LI4Bhva5Q5uik2P-',
    '_score': 0.6544683,
    '_ignored': ['long_message.keyword'],
    '_source': {'headcomponent': 'Generell informasjon om ryggsmerter',
     'educationid': 'Cause of LBP_1\xa0',
     'short_message': 'Rygg- og nakkesmerter har som regel ikke en kjent årsak. Det er derfor svært lite sannsynlig at smertene dine er forårsaket av en alvorlig sykdom eller skade.',
     'link_to_long_message': 'Lær mer',
     'long_message': 'De aller fleste tilfeller av rygg- og nakkesmerter er uspesifikke. Dette betyr at smerten kommer fra muskler og ledd, men at det er vanskelig å identifisere den nøyaktige kilden til problemet. Dette er gode nyheter - du har ingen alvorlig sykdom eller skade i ryggen eller nakken din. ',
     'link_to_tool': 'Lær 

In [10]:
res = es.search(index="data_description",query={"match":{"description_type":"tailoring"}},size=900)
list(map(lambda x: x["_source"],res["hits"]["hits"]))
#["_source"]

[{'questionid': 'BT_pain_average',
  'question': 'Angi hvor mye smerter du har hatt i gjennomsnitt den siste uken',
  'title': 'Smerter',
  'type': 'single',
  'structure': 'horizontal',
  'language': 'nb',
  'options': [{'option': 'ingen smerter',
    'value': '0',
    'optiontype': 'radiobutton',
    'inputtype': 'select',
    'position': 1},
   {'option': '1',
    'value': '1',
    'optiontype': 'radiobutton',
    'inputtype': 'select',
    'position': 2},
   {'option': '2',
    'value': '2',
    'optiontype': 'radiobutton',
    'inputtype': 'select',
    'position': 3},
   {'option': '3',
    'value': '3',
    'optiontype': 'radiobutton',
    'inputtype': 'select',
    'position': 4},
   {'option': '4',
    'value': '4',
    'optiontype': 'radiobutton',
    'inputtype': 'select',
    'position': 5},
   {'option': '5',
    'value': '5',
    'optiontype': 'radiobutton',
    'inputtype': 'select',
    'position': 6},
   {'option': '6',
    'value': '6',
    'optiontype': 'radiobutton'

In [3]:
res = es.search(index="activity", query={'match_all' : {}},size=10000)
res.body["hits"]["hits"]
# list(map(lambda x: x["_id"],res["hits"]["hits"]))

[{'_index': 'activity',
  '_id': '8V4IC48Bvlar5-JnoWLt',
  '_score': 1.0,
  '_source': {'start': 123,
   'end': 12312,
   'type': '1',
   'steps': 4000,
   'userid': 'user'}}]

In [13]:
res = es.index(index='education_queue', 
            document={
                'userid': 'user',
                "educational_items":
                [
                    {'id': 'Changing negative thoughts_7',
                    'priority': 2,
                    'expireDate': 1595019975527,
                    'thisWeek': False,
                    'avoid': False,
                    'excluded': True,
                    'used': False,
                    'usedNumber': 0,
                    'lastUsage': 0,
                    'lastQuiz': False},
                    {'id': 'Changing negative thoughts_6',
                    'priority': 2,
                    'expireDate': 1595019975527,
                    'thisWeek': False,
                    'avoid': False,
                    'excluded': True,
                    'used': False,
                    'usedNumber': 0,
                    'lastUsage': 0,
                    'lastQuiz': False},
                    {'id': 'Changing negative thoughts_5',
                    'priority': 2,
                    'expireDate': 1595019975527,
                    'thisWeek': False,
                    'avoid': False,
                    'excluded': True,
                    'used': False,
                    'usedNumber': 0,
                    'lastUsage': 0,
                    'lastQuiz': False}
                ]
                }
)

print(es.indices.refresh(index='account'))
res

{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}


/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


ObjectApiResponse({'_index': 'education_queue', '_id': 'a7MMIYkBY-fURRVf0Aaa', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1})

In [162]:
res = es.index(index='account', id="stuartgo", 
            document={
                'userid': 'stuartgo',
                'password': '$2b$12$phrVybcm8uyeTl9D/cdYoeZxiOHsjddjitMoYWv8lMVu9bMuY1L2a',
                'country': 'nl',
                'clinician': 'NTNU',
                'rights': 'ROLE_USER',
                'isaccountnonexpired': True,
                'isaccountnonlocked': True,
                'iscredentialsnonexpired': True,
                'isenabled': True}
)

print(es.indices.refresh(index='account'))
res

{'_shards': {'total': 10, 'successful': 5, 'failed': 0}}


/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  >>> conn = connection_from_url('http://google.com/')
/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  >>> conn = connection_from_url('http://google.com/')


ObjectApiResponse({'_index': 'account', '_id': 'stuartgo', '_version': 22, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 37, '_primary_term': 3})

In [ ]:
es.options(ignore_status=[400,404]).indices.delete(index='account')

/home/stuartgo/new_selfback/.venv/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True})

## Old Elasticsearch

In [1]:
from elasticsearch5 import Elasticsearch




es = Elasticsearch("http://localhost:9200")
es.indices.get_alias(index="*")

GET http://localhost:9200/*/_alias [status:N/A request:0.001s]
Traceback (most recent call last):
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/elasticsearch5/connection/http_urllib3.py", line 135, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
  File "/home/stuartgo/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 798, 

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x7f912c113c10>: Failed to establish a new connection: [Errno 111] Connection refused) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x7f912c113c10>: Failed to establish a new connection: [Errno 111] Connection refused)

In [ ]:
language="en"
query={"query":{'match' : {"_type":"education_"+language}}}
query={"query":{'match_all' : {}}}
res = es.search(index="data_description", body=query,size=10000)
print(len(res["hits"]["hits"]))
res["hits"]["hits"]

In [ ]:
#AXK5QIDn3yIeDN3WNAs4

In [ ]:
res = es.search(index="data_description", body={"query":{'match' : {"_type":"exercise_en"}}},size=100)
list(map(lambda x: x["_source"],res["hits"]["hits"]))

In [2]:
res = es.search(index="case", body={"query":{'match_all' : {}}},size=100)
res

{'took': 16,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'failed': 0},
 'hits': {'total': 5834,
  'max_score': 1.0,
  'hits': [{'_index': 'exercise',
    '_type': 't-annegismervik',
    '_id': 'AXK5QIDn3yIeDN3WNAs4',
    '_score': 1.0,
    '_source': {'exerciseid': 'backneck_01_01',
     'performed': 1592245910852,
     'sets': 0,
     'setduration': 0,
     'reps': 0,
     'repsperformed1': 10,
     'repsperformed2': 10,
     'repsperformed3': 10,
     'status': 'completed',
     'reason': None}},
   {'_index': 'exercise',
    '_type': 't-annegismervik',
    '_id': 'AXK5VkEw3yIeDN3WNAs_',
    '_score': 1.0,
    '_source': {'exerciseid': 'glut_01_02',
     'performed': 1592247335769,
     'sets': 0,
     'setduration': 0,
     'reps': 0,
     'repsperformed1': 10,
     'repsperformed2': 10,
     'repsperformed3': 10,
     'status': 'completed',
     'reason': None}},
   {'_index': 'exercise',
    '_type': 'kerstin',
    '_id': 'AXJSQDXF3yIeDN3WM-D_',
    '_score': 1.

In [ ]:
res = es.search(index="account", body={"query":{'match' : {"userid":"stuart126"}}},size=100)
res

In [4]:
response=requests.post(
    "https://back-up.idi.ntnu.no/oauth/token",
    data={'grant_type': 'password',"username":"stuartFinal2","password":"cWbWaN6qgAVjhxdqqx5Ht"},
    verify=False,
    allow_redirects=False,
    auth=("my-trusted-client","secret")
)
token=response.json()["access_token"]
response.json()

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'access_token': '9gQtMI9eHG8MdzUMtZMkWf42ymI',
 'token_type': 'bearer',
 'scope': 'read write trust'}

In [6]:
response=requests.get("https://back-up.idi.ntnu.no/data/achievement/list",verify=False,
                      headers={"Authorization": f"Bearer {token}"}
                      )
response.json()

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{'achievementid': 'SetSleepTool',
  'type': 'general',
  'title': 'Sleep Reminder',
  'description': 'Set a sleep reminder using the Sleep Reminder Tool.',
  'description_achieved': 'Set a sleep reminder using the Sleep Reminder Tool.',
  'image': 'https://dvlxmvtyf5r6m.cloudfront.net/achievements/sleep-reminder.png',
  'language': 'en',
  'goal': 1,
  'goaltext': 'Incomplete.'},
 {'achievementid': 'EducationalMaterialRead1',
  'type': 'educational',
  'title': 'Educational Message Read',
  'description': 'Read an educational message.',
  'description_achieved': 'Read an educational message.',
  'image': 'https://dvlxmvtyf5r6m.cloudfront.net/achievements/educational-read-1.png',
  'language': 'en',
  'goal': 1,
  'goaltext': 'Complete.'},
 {'achievementid': 'EducationalMaterialRead25',
  'type': 'educational',
  'title': 'Educational Message Read',
  'description': 'Read 25 educational messages.',
  'description_achieved': 'Read 25 educational messages.',
  'image': 'https://dvlxmvtyf

In [10]:
response=requests.get("https://back-up.idi.ntnu.no/patient/language",verify=False,
                      headers={"Authorization": f"Bearer {token}"},
                      )
print(response)
response.json()

<Response [200]>


/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{'message': 'en'}

In [55]:
response=requests.post("https://back-up.idi.ntnu.no/patient/plan/next",verify=False,
                      headers={"Authorization": f"Bearer {token}"},
                      json={
                            "exercises_duration": 35,
                            }
                      )
print(response)
response.json()

/home/stuartgo/new_selfback/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'back-up.idi.ntnu.no'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>


{'created': 1687337957813,
 'start': 1687305600000,
 'end': 1687910399999,
 'exercises_duration': 35,
 'history': [{'date': 1687337957813,
   'exercises': [{'exerciseid': 'core_01_02',
     'sets': 3,
     'reps': 10,
     'set_duration_s': -1},
    {'exerciseid': 'flex_01_09', 'sets': 1, 'reps': 1, 'set_duration_s': 30},
    {'exerciseid': 'backneck_01_01',
     'sets': 3,
     'reps': 10,
     'set_duration_s': -1},
    {'exerciseid': 'glut_01_02', 'sets': 3, 'reps': 10, 'set_duration_s': -1},
    {'exerciseid': 'flex_01_03', 'sets': 3, 'reps': 10, 'set_duration_s': -1},
    {'exerciseid': 'ab_01_02', 'sets': 3, 'reps': 10, 'set_duration_s': -1},
    {'exerciseid': 'flex_01_08', 'sets': 1, 'reps': 1, 'set_duration_s': 30}],
   'educations': [{'educationid': 'Family and friends_3',
     'is_quiz': False,
     'is_correct': False},
    {'educationid': 'Problem solving_3',
     'is_quiz': False,
     'is_correct': False},
    {'educationid': 'Stay active_10', 'is_quiz': False, 'is_corre

In [11]:
response=requests.post("http://localhost:30000/token",data={"username":"stuartgo","password":"secret"}
)
token=response.json()["access_token"]
response.json()

NameError: name 'requests' is not defined

In [58]:
response=requests.post("https://localhost:30000/patient/plan/next",verify=False,
                      headers={"Authorization": f"Bearer {token}"},
                      json={
                            "exercises_duration": 35,
                            }
                      )
print(response)
response.json()

SSLError: HTTPSConnectionPool(host='localhost', port=30000): Max retries exceeded with url: /patient/plan/next (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1129)')))

In [20]:
response=requests.get("http://localhost:30000/data/patient/baseline",headers={"Authorization": f"Bearer {token}"}
)
response.json()

ConnectionError: HTTPConnectionPool(host='localhost', port=30000): Max retries exceeded with url: /data/patient/baseline (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f34d6b8e070>: Failed to establish a new connection: [Errno 111] Connection refused'))